In [0]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import set_config; set_config(display='diagram')

# Houses Kaggle Competition (bis 🔥) 

[<img src='https://github.com/lewagon/data-images/blob/master/ML/kaggle-batch-challenge.png?raw=true' width=600>](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

Let's re-use our previous pipeline built in the module `05-07-Ensemble-Methods` and improve final predictions using a Neural Network!

# Re-use already-built preprocessing

## Load data

In [0]:
# Let's load our training dataset
data = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/houses_train_raw.csv")
X = data.drop(columns='SalePrice')
y = data['SalePrice']

# You don't have access to y_yest! Only Kaggle has it.
X_test = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/houses_test_raw.csv")

print(X.shape, y.shape, X_test.shape)

(1460, 80) (1460,) (1459, 80)


## Import preprocessor

You will find the data-preprocessing pipeline that was built in our previous iteration in `utils/preprocessor.py`. 

❓ Run the cell below, and make sure you understand what the pipeline does. Look at the code in `preprocessor.py`

In [0]:
from utils.preprocessor import create_preproc
preproc = create_preproc(X)
preproc

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('numerical_encoder',
                                                  Pipeline(steps=[('knnimputer',
                                                                   KNNImputer()),
                                                                  ('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  ['1stFlrSF', '2ndFlrSF',
                                                   '3SsnPorch', 'BedroomAbvGr',
                                                   'BsmtFinSF1', 'BsmtFinSF2',
                                                   'BsmtFullBath',
                                                   'BsmtHalfBath', 'BsmtUnfSF',
                                                   'EnclosedPorch',
                                                   'Fireplaces', 'FullBath',
                                                   'GarageArea', 'GarageCars...
                                                   'CentralAir', 'Condition1',
                                                   'Condition2', 'Exterior1st',
                                                   'Exterior2nd', 'Foundation',
                                                   'GarageType', 'Heating',
                                                   'HouseStyle', 'LotConfig',
                                                   'MSZoning', 'MasVnrType',
                                                   'MiscFeature',
                                                   'Neighborhood', 'RoofMatl',
                                                   'RoofStyle', 'SaleCondition',
                                                   'SaleType', 'Street',
                                                   'Utilities'])])),
                ('selectpercentile',
                 SelectPercentile(percentile=75,
                                  score_func=<function mutual_info_regression at 0x1310b43a0>))])

❓ Fit the preprocessor on your train set and create your feature matrix `X_preproc` that will be used by the Neural Network

In [0]:
# fit the preprocessor on the train set
preproc.fit(X, y)

# create 
X_preproc = preproc.transform(X)
X_preproc.shape

(1460, 162)

# Your prediction in Keras

This is your first **regression** task with Keras! 
- The cell below contains the compiler and the hyper-parameters we recommend you to start with.
- Kaggle's [rule](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview/evaluation) requires to minimize `rmsle` (Root Mean Square Log Error). As you can see, we were to specify `msle` direcly as loss-function with Keras! Just remember to take square-root of your loss results to read your rmsle metric.
- The best boosted-tree `rmsle` score to beat is around **0.13**

❓ **Question** ❓
- Your responsibility is to build the best model architecture, and to control the epoch number to avoid overfitting.
- We recommand you to create a train/val split upfront to visually control the validation loss thanks to `plot_history`

In [0]:
# Create a train val split here

In [0]:
# def initialize_model():

#     ### YOUR MODEL ARCHITECTURE HERE
#     # model = ...

    
#     # Recommended compilator
#     model.compile(optimizer='adam',
#                   loss='msle', # directly optimize for the squared log error!
#     return model

# model = initialize_model()

# history = model.fit(X_train, y_train,
#                     validation_data=(X_val, y_val),
#                     epochs=100, # Play with this until your validation loss overfit
#                     batch_size=16, # Keep batch size to 16 today
#                     verbose=0)

In [0]:
def plot_history(history):
    plt.plot(np.sqrt(history.history['loss']))
    plt.plot(np.sqrt(history.history['val_loss']))
    plt.title('Model Loss')
    plt.ylabel('MSLE')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='best')
    plt.show()

## SOLUTION BELOW

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_preproc,y,test_size=0.3, shuffle=True)

In [0]:
from tensorflow.keras import Sequential, layers

In [0]:
def initialize_model(X):

    # Model architecture
    model = Sequential()
    model.add(layers.Dense(20, activation='relu', input_dim=X.shape[1]))
    model.add(layers.Dense(15, activation='relu'))
    model.add(layers.Dense(15, activation='relu'))
    model.add(layers.Dense(20, activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    
    
    # Recommended compilator hyperparams
    model.compile(optimizer='adam',
                  loss='msle', # we directly optimize for the kaggle's metric!
    ) 
    
    return model

In [0]:
model = initialize_model(X_train)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 20)                3260      
_________________________________________________________________
dense_6 (Dense)              (None, 15)                315       
_________________________________________________________________
dense_7 (Dense)              (None, 15)                240       
_________________________________________________________________
dense_8 (Dense)              (None, 20)                320       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 21        
Total params: 4,156
Trainable params: 4,156
Non-trainable params: 0
_________________________________________________________________


In [0]:
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val), # create a val set within your train set
                    epochs=500, # Play with this until your validation loss overfit
                    batch_size=16, # Keep batch size to 16 today
                    verbose=1)

Epoch 1/500
64/64 [==============================] - 0s 3ms/step - loss: 111.1317 - val_loss: 83.5250
Epoch 2/500
64/64 [==============================] - 0s 1ms/step - loss: 69.3120 - val_loss: 57.3037
Epoch 3/500
64/64 [==============================] - 0s 1ms/step - loss: 49.4532 - val_loss: 42.2953
Epoch 4/500
64/64 [==============================] - 0s 1ms/step - loss: 36.6350 - val_loss: 31.2773
Epoch 5/500
64/64 [==============================] - 0s 1ms/step - loss: 27.3952 - val_loss: 23.8804
Epoch 6/500
64/64 [==============================] - 0s 1ms/step - loss: 21.3546 - val_loss: 18.9797
Epoch 7/500
64/64 [==============================] - 0s 1ms/step - loss: 17.1965 - val_loss: 15.4593
Epoch 8/500
64/64 [==============================] - 0s 1ms/step - loss: 14.0712 - val_loss: 12.6695
Epoch 9/500
64/64 [==============================] - 0s 2ms/step - loss: 11.4761 - val_loss: 10.2437
Epoch 10/500
64/64 [==============================] - 0s 1ms/step - loss: 9.2633 - val_los

Epoch 83/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0820 - val_loss: 0.0685
Epoch 84/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0812 - val_loss: 0.0679
Epoch 85/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0804 - val_loss: 0.0673
Epoch 86/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0796 - val_loss: 0.0666
Epoch 87/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0788 - val_loss: 0.0660
Epoch 88/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0780 - val_loss: 0.0653
Epoch 89/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0771 - val_loss: 0.0646
Epoch 90/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0763 - val_loss: 0.0639
Epoch 91/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0754 - val_loss: 0.0632
Epoch 92/500
64/64 [==============================] - 0s 2ms/step - loss: 0.0745 - val_loss: 0.0625


64/64 [==============================] - 0s 1ms/step - loss: 0.0257 - val_loss: 0.0257
Epoch 165/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0254 - val_loss: 0.0255
Epoch 166/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0253 - val_loss: 0.0258
Epoch 167/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0250 - val_loss: 0.0252
Epoch 168/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0246 - val_loss: 0.0250
Epoch 169/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0244 - val_loss: 0.0248
Epoch 170/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0241 - val_loss: 0.0246
Epoch 171/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0239 - val_loss: 0.0244
Epoch 172/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0236 - val_loss: 0.0243
Epoch 173/500
64/64 [==============================] - 0s 2ms/step - loss: 0.0235 - val_loss: 0.0241
Epoc

64/64 [==============================] - 0s 1ms/step - loss: 0.0139 - val_loss: 0.0208
Epoch 246/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0139 - val_loss: 0.0207
Epoch 247/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0138 - val_loss: 0.0203
Epoch 248/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0138 - val_loss: 0.0203
Epoch 249/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0137 - val_loss: 0.0202
Epoch 250/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0136 - val_loss: 0.0204
Epoch 251/500
64/64 [==============================] - 0s 2ms/step - loss: 0.0136 - val_loss: 0.0203
Epoch 252/500
64/64 [==============================] - 0s 2ms/step - loss: 0.0135 - val_loss: 0.0203
Epoch 253/500
64/64 [==============================] - 0s 2ms/step - loss: 0.0135 - val_loss: 0.0203
Epoch 254/500
64/64 [==============================] - 0s 2ms/step - loss: 0.0134 - val_loss: 0.0204
Epoc

64/64 [==============================] - 0s 1ms/step - loss: 0.0108 - val_loss: 0.0229
Epoch 327/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0108 - val_loss: 0.0225
Epoch 328/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0107 - val_loss: 0.0226
Epoch 329/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0107 - val_loss: 0.0226
Epoch 330/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0107 - val_loss: 0.0228
Epoch 331/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0107 - val_loss: 0.0235
Epoch 332/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0107 - val_loss: 0.0226
Epoch 333/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0107 - val_loss: 0.0232
Epoch 334/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0107 - val_loss: 0.0242
Epoch 335/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0106 - val_loss: 0.0236
Epoc

64/64 [==============================] - 0s 1ms/step - loss: 0.0096 - val_loss: 0.0278
Epoch 408/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0095 - val_loss: 0.0283
Epoch 409/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0095 - val_loss: 0.0287
Epoch 410/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0095 - val_loss: 0.0289
Epoch 411/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0095 - val_loss: 0.0278
Epoch 412/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0094 - val_loss: 0.0297
Epoch 413/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0095 - val_loss: 0.0286
Epoch 414/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0094 - val_loss: 0.0294
Epoch 415/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0095 - val_loss: 0.0301
Epoch 416/500
64/64 [==============================] - 0s 1ms/step - loss: 0.0094 - val_loss: 0.0301
Epoc

64/64 [==============================] - 0s 2ms/step - loss: 0.0087 - val_loss: 0.0568
Epoch 489/500
64/64 [==============================] - 0s 2ms/step - loss: 0.0087 - val_loss: 0.0718
Epoch 490/500
64/64 [==============================] - 0s 2ms/step - loss: 0.0086 - val_loss: 0.0825
Epoch 491/500
64/64 [==============================] - 0s 2ms/step - loss: 0.0087 - val_loss: 0.0944
Epoch 492/500
64/64 [==============================] - 0s 2ms/step - loss: 0.0086 - val_loss: 0.0849
Epoch 493/500
64/64 [==============================] - 0s 2ms/step - loss: 0.0087 - val_loss: 0.0573
Epoch 494/500
64/64 [==============================] - 0s 2ms/step - loss: 0.0086 - val_loss: 0.0838
Epoch 495/500
64/64 [==============================] - 0s 2ms/step - loss: 0.0086 - val_loss: 0.1095
Epoch 496/500
64/64 [==============================] - 0s 2ms/step - loss: 0.0086 - val_loss: 0.1257
Epoch 497/500
64/64 [==============================] - 0s 2ms/step - loss: 0.0085 - val_loss: 0.0519
Epoc

In [0]:
# val RMSE at last epoch
model.evaluate(X_val, y_val)**0.5

14/14 [==============================] - 0s 1ms/step - loss: 0.0261


0.16165394751625214

In [0]:
# Minimal val RMSE reached amongst all epochs
min(history.history['val_loss'])**0.5

0.14195845779202246

❓ **Question** ❓
- Are you satisfied with your score?
- Before you publish it, ask yourself if you could really trust it ? Has it been cross-validated? 
- Feel free to cross-validate it manually with a *for loop* in Python if you want before submitting to Kaggle

In [0]:
def evaluate_model(X,y,train_index,val_index):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    model = initialize_model(X_train)
    history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val), 
                    epochs=500,
                    batch_size=16,
                    verbose=0)
    return pd.DataFrame({
        'rmsle_final_epoch': [model.evaluate(X_val, y_val)**0.5],
        'rmsle_min': [min(history.history['val_loss'])**0.5]
    })

In [0]:
from sklearn.model_selection import KFold
cv = 5
kf = KFold(n_splits=cv, shuffle=True)

In [0]:
results = []

for train_index, val_index in kf.split(X_preproc):
    results.append(evaluate_model(X_preproc, y, train_index, val_index))
    
pd.concat(results, axis=0)

10/10 [==============================] - 0s 1ms/step - loss: 0.0187


,rmsle_final_epoch,rmsle_min
0,0.141220,0.140153
0,0.143595,0.138378
0,0.151638,0.141255
0,12.023061,12.023061
0,0.136732,0.129074


**BONUS SOLUTION**: Multiprocessing using [dask](https://docs.dask.org/en/latest/delayed.html) and 8 CPU cores to mimic sklearn's `n_jobs=-1`

In [0]:
!pip install --quiet dask

In [0]:

from dask import delayed

f = delayed(evaluate_model)

results = delayed(
    [f(X_preproc, y, train_index, val_index) 
     for (train_index, val_index) in kf.split(X_preproc)]
).compute(
    scheduler='processes',
    num_workers=8)

pd.concat(results, axis=0)

2021-08-09 12:13:39.644838: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-09 12:13:39.793451: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


10/10 [==============================] - 0s 518us/step - loss: 0.0227


,rmsle_final_epoch,rmsle_min
0,0.150564,0.140305
0,0.193191,0.139178
0,0.133644,0.130848
0,0.145214,0.141758
0,0.169600,0.161516


TODO: multiprocessing with default python libray
References [here](https://towardsdatascience.com/speeding-up-and-perfecting-your-work-using-parallel-computing-8bc2f0c073f8) and [here](https://johaupt.github.io/python/parallel%20processing/cross-validation/multiprocessing_cross_validation.html)

In [0]:
# import multiprocessing as mp
# pool = mp.Pool(processes=2) #mp.cpu_count()-1)

# results = []
# def log_result(x):
#     results.append(x)
    
# for train_index, val_index in kf.split(X_preproc):
#     pool.apply_async(
#         evaluate_model,
#         args=(X, y, train_index, val_index),
#         callback = log_result)

# # Close the pool for new tasks
# pool.close()

# # Wait for all tasks to complete at this point
# pool.join()

# result = pd.concat(results, axis=0)

# 🏅FINAL SUBMISSION

Predict the house prices of your test set and submit your results to kaggle! Be careful with the format of the exported `.csv`.

In [0]:
X_test = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/houses_test_raw.csv")

In [0]:
X_test_preproc = preproc.transform(X_test)
predictions = model.predict(X_test_preproc)
predictions

array([[126270.94],
       [157179.14],
       [180056.66],
       ...,
       [166608.08],
       [132417.62],
       [203446.42]], dtype=float32)

In [0]:
results = pd.concat([X_test["Id"], pd.Series(predictions[:,0], name="SalePrice")], axis=1)
results

,Id,SalePrice
0,1461,126270.937500
1,1462,157179.140625
2,1463,180056.656250
3,1464,188388.671875
4,1465,187704.234375
...,...,...
1454,2915,82110.828125
1455,2916,69048.164062
1456,2917,166608.078125
1457,2918,132417.625000


In [0]:
# Export to Kaggle format submission and submit it online!
results.to_csv("submission_final.csv", header=True, index=False)